In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("noeyislearning/framingham-heart-study")

print("Path to dataset files:", path)

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/morning/.cache/kagglehub/datasets/noeyislearning/framingham-heart-study/versions/1


In [2]:
import os
os.listdir(path)

['framingham_heart_study.csv']

In [3]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv(f'{path}/framingham_heart_study.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
df.shape

(4240, 16)

In [6]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3658 entries, 0 to 3657
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3658 non-null   int64  
 1   age              3658 non-null   int64  
 2   education        3658 non-null   float64
 3   currentSmoker    3658 non-null   int64  
 4   cigsPerDay       3658 non-null   float64
 5   BPMeds           3658 non-null   float64
 6   prevalentStroke  3658 non-null   int64  
 7   prevalentHyp     3658 non-null   int64  
 8   diabetes         3658 non-null   int64  
 9   totChol          3658 non-null   float64
 10  sysBP            3658 non-null   float64
 11  diaBP            3658 non-null   float64
 12  BMI              3658 non-null   float64
 13  heartRate        3658 non-null   float64
 14  glucose          3658 non-null   float64
 15  TenYearCHD       3658 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 457.4 KB


In [7]:
df['male'].unique()

array([1, 0])

In [8]:
y = df['TenYearCHD'].values
X = df.drop('TenYearCHD', axis=1)

In [9]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(3658, 15)
(3658, 12)


In [10]:
# build mask
gender_0_mask = df['male'] == 0
gender_1_mask = df['male'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  2035
Male:  1623


In [11]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [12]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.000000  1.000000  0.000000  1.000000       0     181   
1      2  Female  0.000000  0.988889  0.011111  1.000000       0     178   
2      3  Female  0.033333  1.000000  0.000000  0.966667       1     174   
3      4  Female  0.000000  1.000000  0.000000  1.000000       0     187   
4      5  Female  0.000000  1.000000  0.000000  1.000000       0     182   
5      6  Female  0.000000  1.000000  0.000000  1.000000       0     179   
6      7  Female  0.000000  1.000000  0.000000  1.000000       0     171   
7      8  Female  0.000000  0.994286  0.005714  1.000000       0     174   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
results_df = pd.concat(results_list, ignore_index=True)

In [14]:
result_path = './results/K27_result.xlsx'
results_df.to_excel(result_path, index=False)

In [15]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.000000,1.000000,0.000000,1.000000,0,181,0,23,...,2,21,0.260870,0.939227,0.060773,0.739130,6,170,11,17
1,2,Female,0.000000,0.988889,0.011111,1.000000,0,178,2,24,...,11,23,0.208333,0.861111,0.138889,0.791667,5,155,25,19
2,3,Female,0.033333,1.000000,0.000000,0.966667,1,174,0,29,...,2,28,0.333333,0.867816,0.132184,0.666667,10,151,23,20
3,4,Female,0.000000,1.000000,0.000000,1.000000,0,187,0,17,...,2,16,0.235294,0.882353,0.117647,0.764706,4,165,22,13
4,5,Female,0.000000,1.000000,0.000000,1.000000,0,182,0,22,...,4,19,0.545455,0.901099,0.098901,0.454545,12,164,18,10


In [16]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(21.5), pvalue=np.float64(0.023606658933052256))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(49.0), pvalue=np.float64(0.9568829891119314))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(36.0), pvalue=np.float64(0.3067609561706547))
DT -TPR: MannwhitneyuResult(statistic=np.float64(24.0), pvalue=np.float64(0.05354193923094535))
DT - FPR: TtestResult(statistic=np.float64(-2.6712955432776546), pvalue=np.float64(0.015571311214377992), df=np.float64(18.0))
DT - FN/FP: TtestResult(statistic=np.float64(-0.6120910488902191), pvalue=np.float64(0.5481321974846964), df=np.float64(18.0))
RF -TPR: TtestResult(statistic=np.float64(-2.3330441350889757), pvalue=np.float64(0.039250665560712335), df=np.float64(11.209981271310234))
RF - FPR: MannwhitneyuResult(statistic=np.float64(15.0), pvalue=np.float64(0.009030399155626945))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(71.5), pvalue=np.float64(0.11214040300651025))
LR -TPR: Mannwhitne